<img src='./img/LogoWekeo_Copernicus_RGB_0.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='20%'></img>

<a href="./00_index.ipynb"><< Index</a><br>
<a href="./20_Sentinel5P_TROPOMI_NO2_L2_retrieve.ipynb"><< 20 - Sentinel-5P NO<sub>2</sub> - Nitrogen Dioxide - Retrieve</a>

<div class="alert alert-block alert-warning">
<b>LOAD, BROWSE AND VISUALIZE</b></div>

# Copernicus Sentinel-5 Precursor (Sentinel-5P) - NO<sub>2</sub>

The subsequent example introduces you to Sentinel-5P data in general and the total column of NO<sub>2</sub> sensed by Sentinel-5P in specific. NO<sub>2</sub> is useful for monitoring air pollution. The example is based on elevated nitrogen dioxide levels in Europe which occurred in February 2021.


#### Module outline:
* [1 - Load and browse Sentinel-5P TROPOMI data](#load_s5P)
* [2 - Create a geographical subset](#geographical_subset)
* [3 - Visualize Sentinel-5P NO<sub>2</sub> data](#visualize_s5P)

#### Load required libraries

In [ ]:
%matplotlib inline
import os
import xarray as xr
import numpy as np
import netCDF4 as nc

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

import subprocess
import shlex
import zipfile

from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
GeoAxes._pcolormesh_patched = Axes.pcolormesh

#### Load helper functions

In [ ]:
%run ./functions.ipynb

<hr>

### Decompressing Sentinel-3 OLCI Level 1B Full Resolution data

The Sentinel-5P TROPOMI data could be not available on the machine. In this case there is the method to download them to [20 - Sentinel-5 TROPOMI NO2_Level 2 Retrieve](./20_Sentinel5P_TROPOMI_NO2_L2_retrieve.ipynb).

Also the user must extract the data before load them, if they are provided as a zip archive. If you have an already decompressed data this step is not necessary and the data can be directly loaded.

In [ ]:
#rename S5 file
download_dir_path = os.getcwd()

for item in os.listdir(download_dir_path):
    if not os.path.isdir(item):
        cmd = shlex.split('file --mime-type {0}'.format(item))
        result = subprocess.check_output(cmd)
        mime_type = result.split()[-1]
        if item.startswith('S5P_OFFL_L2_') and not item.endswith('.zip') and mime_type == b'application/zip':
            os.rename(os.path.join(download_dir_path, item), os.path.join(download_dir_path, item +".zip"))

In case you have permission issues when renaming the archive with the code above, please manually do that by right-clicking the file, then rename, then add .zip to the end of the filename.

For example:
- From: S5P_OFFL_L2__NO2____20210205T104439_20210205T122609_17182_01_010400_20210207T042548   
- To: S5P_OFFL_L2__NO2____20210205T104439_20210205T122609_17182_01_010400_20210207T042548.zip

In [ ]:
extension = ".zip"

for item in os.listdir(download_dir_path): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.join(download_dir_path, item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(download_dir_path) # extract file to dir
        zip_ref.close() # close file
        

## <a id="load_s5P"></a>Load and browse Sentinel-5P data

A Sentinel-5P file is organised in two groups: `PRODUCT` and `METADATA`. The `PRODUCT` group stores the main data fields of the product, including `latitude`, `longitude` and the variable itself. The `METADATA` group provides additional metadata items.

Sentinel-5P variables have the following dimensions:
* `scanline`: the number of measurements in the granule / along-track dimension index
* `ground_pixel`: the number of spectra in a measurement / across-track dimension index
* `time`: time reference for the data
* `corner`: pixel corner index
* `layer`: this dimension indicates the vertical grid of profile variables

Sentinel-5P TROPOMI data is disseminated in `netCDF`. You can load a `netCDF` file with the `open_dataset()` function of the xarray library. In order to load the variable as part of a Sentinel-5P data files, you have to specify the following keyword arguments: 
- `group='PRODUCT'`: to load the `PRODUCT` group

Let us load a Sentinel-5P TROPOMI data file as `xarray.Dataset` from 5 February 2021 and inspect the data structure:


In [ ]:
s5P = xr.open_dataset('./S5P_OFFL_L2__NO2____20210205T104439_20210205T122609_17182_01_010400_20210207T042548/S5P_OFFL_L2__NO2____20210205T104439_20210205T122609_17182_01_010400_20210207T042548.nc', group='PRODUCT')
s5P

You see that a Sentinel-5P data file consists of eight dimensions and twelve data variables:

* **Dimensions**:
  * `scanline` 
  * `ground_pixel`
  * `time`
  * `corner`
  * `polynomial_exponents`
  * `intensity_offset_polynomial_exponents`
  * `layer`
  * `vertices`

* **Data variables**:
  * `delta_time`: the offset of individual measurements within the granule, given in milliseconds
  * `time_utc`: valid time stamp of the data
  * `qa_value`: quality descriptor, varying between 0 (nodata) and 1 (full quality data).
  * `nitrogendioxide_tropospheric_column`: Vertically integrated NO<sub>2</sub> column density
  * `nitrogendioxide_tropospheric_column_precision`: Standard error of the vertically integrated NO<sub>2</sub> column
  
  As well as a few other variables:
  * `nitrogendioxide_tropospheric_column_precision_kernel`
  * `averaging_kernel`
  * `air_mass_factor_troposphere`
  * `air_mass_factor_total`
  * `tm5_tropopause_layer_index`
  * `tm5_constant_a`
  * `tm5_constant_b`

You can specify one variable of interest and get more detailed information about the variable. E.g. `nitrogendioxide_total_column` is the atmosphere mole content of NO<sub>2</sub>, has the unit `mol m-2` (which means `mol per m2`), and has three dimensions, `time`, `scanline` and `groundpixel` respectively.

In [ ]:
s5P_no2 = s5P['nitrogendioxide_tropospheric_column']
s5P_no2

You can do this for the available variables, but also for the dimensions latitude and longitude.

In [ ]:
latitude = s5P_no2.latitude
latitude

<br>

In [ ]:
longitude = s5P_no2.longitude
longitude

You can retrieve the array values of the variable with squared bracket: `[:,:,:]`. One single time step can be selected by specifying one value of the time dimension, e.g. `[0,:,:]`.

In [ ]:
s5P_no2_0502 = s5P_no2[0,:,:]
s5P_no2_0502

The attributes of the data array hold the entry `multiplication_factor_to_convert_to_molecules_percm2`, which is a conversion factor that has to be applied to convert the data from `mol per m2` to `molecules per cm2`.


In [ ]:
conversion_factor = s5P_no2_0502.multiplication_factor_to_convert_to_molecules_percm2
conversion_factor

Additionally, you can save the attribute `longname`, which you can make use of when visualizing the data.

In [ ]:
longname = s5P_no2_0502.long_name
longname

## <a id='geographical_subset'></a>Create  a geographical subset

You can zoom into a region by specifying a `bounding box` of interest. Let's set the extent to Europe with the following bounding box information:

In [ ]:
latmin = 28.
latmax = 71.
lonmin = -22.
lonmax = 43

You can use the function [generate_geographical_subset()](./functions.ipynb#generate_geographical_subset) to subset an xarray DataArray based on a given bounding box.

In [ ]:
s5P_no2_subset = generate_geographical_subset(s5P_no2_0502, latmin, latmax, lonmin, lonmax)
s5P_no2_subset

<br>

## <a id="plotting_s5P"></a>Plotting example - Sentinel-5P TROPOMI data

You can plot data arrays of type `numpy` with matplotlib's `pcolormesh` function. In combination with the library [cartopy](https://scitools.org.uk/cartopy/docs/latest/), you can produce high-quality maps. 

In order to make it easier to visualize the NO<sub>2</sub> values, we apply the conversion factor to the DataArray. This converts the NO<sub>2</sub> values from mol per m<sup>2</sup> to molecules per cm<sup>2</sup>.

In [ ]:
s5P_no2_converted = s5P_no2_subset*conversion_factor
s5P_no2_converted

The next step is to visualize the dataset. You can use the function [visualize_pcolormesh](../functions.ipynb#visualize_pcolormesh), which makes use of matploblib's function `pcolormesh` and the [Cartopy](https://scitools.org.uk/cartopy/docs/latest/) library.

With `?visualize_pcolormesh` you can open the function's docstring to see what keyword arguments are needed to prepare your plot.

In [ ]:
?visualize_pcolormesh

Now, let us apply the [visualize_pcolormesh](./functions#visualize_pcolormesh) function and visualize the vertically integrated NO<sub>2</sub> column sensored from the Sentinel-5P satellite on 5 February 2021.

Note: Multiplying the `DataArray` values with `1e-15` improves the readibility of the map legend.

In [ ]:
visualize_pcolormesh(data_array=s5P_no2_converted*1e-15,
                     longitude=s5P_no2_converted.longitude,
                     latitude=s5P_no2_converted.latitude,
                     projection=ccrs.PlateCarree(),
                     color_scale='viridis',
                     unit='*1e-15 molecules per cm2',
                     long_name=longname + ' ' + str(s5P_no2_converted.time.data),
                     vmin=0,
                     vmax=35,
                     lonmin=lonmin,
                     lonmax=lonmax,
                     latmin=latmin,
                     latmax=latmax,
                     set_global=False)

<br>

<a href="./00_index.ipynb"><< Index</a><br>
<a href="./20_Sentinel5P_TROPOMI_NO2_L2_retrieve.ipynb"> << 20 - Sentinel-5P NO<sub>2</sub> - Nitrogen Dioxide - Retrieve</a>

<hr>

<p><img src='./img/all_partners_wekeo.png' align='left' alt='Logo EU Copernicus' width='100%'></img><p>